In [2]:
#Import libraries for feature engineering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression, SelectKBest
import warnings

# Configure settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("🔧 FEATURE ENGINEERING SETUP")
print("=" * 35)
print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🤖 Scikit-learn available for feature selection")

🔧 FEATURE ENGINEERING SETUP
✅ Libraries imported successfully!
📊 Pandas version: 2.2.3
🤖 Scikit-learn available for feature selection


In [22]:
#Load cleaned data (from data processing step)
df = pd.read_csv('../data/processed/prepocessed_daily_data.csv')  # Will be updated to use processed data
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)

In [23]:
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])
df['day_length_hours'] = df['sunset'] - df['sunrise']
df = df.drop(columns=['sunrise', 'sunset'])
df['day_length_hours'] = df['day_length_hours'].dt.total_seconds() / 3600.0

df['target'] = df['temp'].shift(-5)
df = df[~df['target'].isna()].copy()

In [24]:
df["temp_solar_interaction"] = df["temp"] * df["solarradiation"]
df["uv_temp_interaction"] = df["uvindex"] * df["temp"]
df['temp_cloudcover_interaction'] = df['temp'] * df['cloudcover']
df['temp_sealevelpressure_interaction'] = df['temp'] * df['sealevelpressure']

In [25]:
# Create lagging features
def create_lag_features(df, cols, lags):
    for col in cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
    return df

# Specify columns and lags
# Get all numerical columns
computing_columns = df.drop(columns=['year', 'month', 'day', 'day_of_year', 'season', 'is_rainy', 'target']).columns

lag_steps = [1, 2, 3, 5, 7, 10, 14]  # Example lag steps

# Apply lagging features before handling rolling horizons
df = create_lag_features(df, computing_columns, lag_steps)

# Function to compute rolling mean and percentage change
def compute_rolling(df, horizon, col):
    label = f"rolling_{horizon}_{col}"
    df[label] = df[col].rolling(horizon, min_periods=horizon).mean()  # Ensure full horizon is used
    df[f"{label}_change"] = df[col] - df[label]
    return df

# Compute rolling features for specified horizons
rolling_horizons = [3, 7, 14]  # Rolling windows of 3, 7, 14 days
for horizon in rolling_horizons:
    for col in computing_columns:
        df = compute_rolling(df, horizon, col)

# Drop rows with NaN values caused by rolling horizons
df = df.iloc[14:]
# Verify no NaN values exist
nan_summary = df.isna().sum()
print("Summary of NaN values in each column after handling rolling horizons and lagging:")
print(nan_summary[nan_summary > 0])

if df.isna().any().any():
    print("\nThe dataframe contains NaN values.")
else:
    print("\nThe dataframe does not contain any NaN values.")

Summary of NaN values in each column after handling rolling horizons and lagging:
Series([], dtype: int64)

The dataframe does not contain any NaN values.


In [26]:
#Months and days average
def expand_mean(df):
    return df.expanding(1).mean()

for col in computing_columns:
    df[f"month_avg_{col}"] = df[col].groupby(df.index.month, group_keys=False).apply(expand_mean)
    df[f"day_avg_{col}"] = df[col].groupby(df.index.day_of_year, group_keys=False).apply(expand_mean)
    df[f"year_avg_{col}"] = df[col].groupby(df.index.year, group_keys=False).apply(expand_mean)
    df[f"season_avg_{col}"] = df[col].groupby(df['season'], group_keys=False).apply(expand_mean)
    df["month_max_temp"] = df['temp'].groupby(df.index.month, group_keys=False).cummax()
    df["month_min_temp"] = df['temp'].groupby(df.index.month, group_keys=False).cummin()

df["temp_volatility_7"] = df["temp"].rolling(7).std()
df["temp_volatility_30"] = df["temp"].rolling(30).std()
df["temp_spike_flag"] = (df["temp"] - df["temp"].shift(1)).abs() > 5
df["temp_anomaly_vs_month_avg"] = df["temp"] - df["month_avg_temp"]
df["temp_anomaly_vs_season_avg"] = df["temp"] - df["season_avg_temp"]

df["pressure_trend_3d"] = df["sealevelpressure"] - df["sealevelpressure"].shift(3)
df["pressure_trend_7d"] = df["sealevelpressure"] - df["sealevelpressure"].shift(7)

In [27]:
df.shape

(3635, 502)

In [16]:
df.to_csv('../data/processed/feature_engineering_daily_data2.csv')

In [17]:
#Import library for model training
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import numpy as np
import seaborn as sns
from xgboost import XGBRegressor
from scipy.stats import uniform, randint
import optuna
pd.set_option('display.max_columns', None)

#Loading processed data
df = pd.read_csv('../data/processed/feature_engineering_daily_data2.csv', index_col='datetime')

In [28]:
X = df.drop(columns=['target'])
y = df['target']

cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# ---------- 1) Khóa schema OHE toàn cục ----------
ohe_template = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_template.fit(X[cat_cols])
fixed_categories = ohe_template.categories_

def make_preprocessor():
    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore',
                              sparse_output=False,
                              categories=fixed_categories))
    ])
    return ColumnTransformer([
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ], remainder='drop')

# ---------- 2) Cố định mặt nạ VarianceThreshold từ FOLD ĐẦU ----------
tscv = TimeSeriesSplit(n_splits=5)
first_train_idx, first_test_idx = next(tscv.split(X))

# Fit preprocessor + VT trên fold đầu
preprocessor = make_preprocessor()
preprocessor.fit(X.iloc[first_train_idx], y.iloc[first_train_idx])
X_first_train_trans = preprocessor.transform(X.iloc[first_train_idx])
vt = VarianceThreshold(threshold=0.0).fit(X_first_train_trans)
vt_support_mask = vt.get_support()
feat_names_all = preprocessor.get_feature_names_out()
feat_names_after_vt = feat_names_all[vt_support_mask]

def apply_vt_mask(X_mat, mask=vt_support_mask):
    return X_mat[:, mask]

In [489]:
print("BẮT ĐẦU TUNING XGBoost với Optuna...")

# Dùng 70-80% dữ liệu đầu để tuning
train_ratio = 0.8
split_idx = int(train_ratio * len(X))
X_tune, y_tune = X.iloc[:split_idx], y.iloc[:split_idx]

# Preprocess cố định
prep_tune = make_preprocessor()
prep_tune.fit(X_tune, y_tune)
X_tune_t = apply_vt_mask(prep_tune.transform(X_tune))

# Hàm objective cho Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'n_jobs': -1,
        'tree_method': 'hist'
    }

    tscv = TimeSeriesSplit(n_splits=5)
    all_r2 = []
    for tr_idx, te_idx in tscv.split(X_tune):
        X_train, X_test = X_tune.iloc[tr_idx], X_tune.iloc[te_idx]
        y_train, y_test = y_tune.iloc[tr_idx], y_tune.iloc[te_idx]

        X_train_t = apply_vt_mask(prep_tune.transform(X_train))
        X_test_t = apply_vt_mask(prep_tune.transform(X_test))

        model = XGBRegressor(**params)
        model.fit(X_train_t, y_train)
        y_pred = model.predict(X_test_t)
        r2 = r2_score(y_test, y_pred)
        all_r2.append(r2)

    return np.mean(all_r2)

# Tạo study và tối ưu hóa
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=35)  # Tương đương n_iter=100

# Lấy tham số tốt nhất
best_params = study.best_params
print("HOÀN THÀNH TUNING! R2 tốt nhất:", study.best_value)
print("Tham số tốt nhất:", best_params)

[I 2025-10-27 00:20:13,373] A new study created in memory with name: no-name-2afe0506-b4aa-462b-8f01-80387ed96659


BẮT ĐẦU TUNING XGBoost với Optuna...


[I 2025-10-27 00:20:45,095] Trial 0 finished with value: 0.6841546321170424 and parameters: {'n_estimators': 177, 'max_depth': 12, 'learning_rate': 0.08952457031201479, 'subsample': 0.8125433978634484, 'colsample_bytree': 0.7864971793172957, 'min_child_weight': 8, 'reg_alpha': 8.938554972762976, 'reg_lambda': 8.94144419417929}. Best is trial 0 with value: 0.6841546321170424.
[I 2025-10-27 00:21:27,783] Trial 1 finished with value: 0.6855786113860656 and parameters: {'n_estimators': 375, 'max_depth': 7, 'learning_rate': 0.10284749546001738, 'subsample': 0.5531401147361343, 'colsample_bytree': 0.8600006752638711, 'min_child_weight': 10, 'reg_alpha': 6.619475160118753, 'reg_lambda': 5.1420312695275685}. Best is trial 1 with value: 0.6855786113860656.
[I 2025-10-27 00:24:51,276] Trial 2 finished with value: 0.6832774509727118 and parameters: {'n_estimators': 816, 'max_depth': 12, 'learning_rate': 0.06564928958505832, 'subsample': 0.5061200810361614, 'colsample_bytree': 0.7867643784889164, 

KeyboardInterrupt: 

In [20]:
best_params = {'n_estimators': 487, 'max_depth': 7, 'learning_rate': 0.011293849270387353, 'subsample': 0.6871705314157253, 'colsample_bytree': 0.5317995526559421, 'min_child_weight': 8, 'reg_alpha': 0.6342918304930286, 'reg_lambda': 1.235926349200246}

In [29]:
# ---------- 3) CV vòng 1: DÙNG MODEL ĐÃ TUNED (KHÔNG TUNING LẠI) ----------
tscv = TimeSeriesSplit(n_splits=5)
all_rmse, all_mae, all_r2 = [], [], []
perm_importances_list = []

print("\n=== CV VÒNG 1: DÙNG MODEL ĐÃ TUNED + Permutation Importance ===")
for fold, (tr_idx, te_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train, y_test = y.iloc[tr_idx], y.iloc[te_idx]

    preprocessor = make_preprocessor()
    preprocessor.fit(X_train, y_train)
    X_train_t = apply_vt_mask(preprocessor.transform(X_train))
    X_test_t  = apply_vt_mask(preprocessor.transform(X_test))

    # DÙNG MODEL ĐÃ TUNED
    model = XGBRegressor(**best_params, random_state=42, n_jobs=-1, tree_method='hist')
    model.fit(X_train_t, y_train)
    y_pred = model.predict(X_test_t)

    # Metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    all_rmse.append(rmse); all_mae.append(mae); all_r2.append(r2)

    # Permutation Importance
    perm = permutation_importance(model, X_test_t, y_test, n_repeats=3, random_state=42, n_jobs=-1)
    perm_importances_list.append(perm.importances_mean)

    print(f"Fold {fold}: RMSE={rmse:.3f}  R2={r2:.3f}")

print("\n=== KẾT QUẢ TỔNG KẾT CV VÒNG 1 ===")
print(f"Avg RMSE: {np.mean(all_rmse):.3f} ± {np.std(all_rmse):.3f}")
print(f"Avg MAE: {np.mean(all_mae):.3f} ± {np.std(all_mae):.3f}")
print(f"Avg R2: {np.mean(all_r2):.3f} ± {np.std(all_r2):.3f}")


=== CV VÒNG 1: DÙNG MODEL ĐÃ TUNED + Permutation Importance ===
Fold 1: RMSE=2.831  R2=0.692
Fold 2: RMSE=2.601  R2=0.703
Fold 3: RMSE=2.586  R2=0.757
Fold 4: RMSE=2.621  R2=0.736
Fold 5: RMSE=2.337  R2=0.771

=== KẾT QUẢ TỔNG KẾT CV VÒNG 1 ===
Avg RMSE: 2.595 ± 0.157
Avg MAE: 2.050 ± 0.114
Avg R2: 0.732 ± 0.030


In [30]:
# === CHỌN TOP-K TỪ TRUNG BÌNH ===
avg_perm = np.mean(perm_importances_list, axis=0)
order = np.argsort(avg_perm)[::-1]
cumsum = np.cumsum(avg_perm[order]); cumsum /= cumsum[-1]
top_k = 100
top_idx = order[:top_k]

print(f"\nTop-{top_k} features được chọn từ model đã tuning.")


Top-100 features được chọn từ model đã tuning.


In [31]:
import optuna
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Giả sử các hàm make_preprocessor, apply_vt_mask, select_topk, X, y, top_idx, top_k đã được định nghĩa

def select_topk(X_transformed, top_idx=top_idx):
    return X_transformed[:, top_idx] if isinstance(X_transformed, np.ndarray) else X_transformed.iloc[:, top_idx]

# Hàm objective cho Optuna
def objective(trial):
    # Định nghĩa không gian tham số
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'n_jobs': -1,
        'tree_method': 'hist'
    }

    # Khởi tạo danh sách để lưu RMSE và R2
    tscv = TimeSeriesSplit(n_splits=5)
    all_rmse, all_r2 = [], []

    # Cross-validation
    for tr_idx, te_idx in tscv.split(X):
        X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
        y_train, y_test = y.iloc[tr_idx], y.iloc[te_idx]

        # Preprocessing
        preprocessor = make_preprocessor()
        preprocessor.fit(X_train, y_train)
        X_train_t = select_topk(apply_vt_mask(preprocessor.transform(X_train)), top_idx)
        X_test_t = select_topk(apply_vt_mask(preprocessor.transform(X_test)), top_idx)

        # Huấn luyện mô hình
        model = XGBRegressor(**params)
        model.fit(X_train_t, y_train)
        y_pred = model.predict(X_test_t)

        # Tính toán metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        all_rmse.append(rmse)
        all_r2.append(r2)

    # Trả về giá trị trung bình R2 (Optuna tối ưu hóa theo hướng tối đa)
    return np.mean(all_r2)

# Tạo study và tối ưu hóa
study = optuna.create_study(direction='maximize')  # Tối đa hóa R2
study.optimize(objective, n_trials=100)  # Thử 100 tổ hợp tham số

# Lấy tham số tốt nhất
best_params = study.best_params
print("Best parameters:", best_params)
print(f"Best R2: {study.best_value:.3f}")

[I 2025-10-27 09:15:25,496] A new study created in memory with name: no-name-d2b3897d-2986-4061-be1d-ee4984988aa9
[I 2025-10-27 09:15:40,081] Trial 0 finished with value: 0.719422500697122 and parameters: {'n_estimators': 933, 'max_depth': 8, 'learning_rate': 0.02398396562062577, 'subsample': 0.6323781317397077, 'colsample_bytree': 0.9999418263101227, 'min_child_weight': 9, 'reg_alpha': 9.931189638306169, 'reg_lambda': 0.9417948215149696}. Best is trial 0 with value: 0.719422500697122.
[I 2025-10-27 09:15:45,230] Trial 1 finished with value: 0.6898208443397305 and parameters: {'n_estimators': 642, 'max_depth': 8, 'learning_rate': 0.2363802949508219, 'subsample': 0.8643734905056762, 'colsample_bytree': 0.705557947955507, 'min_child_weight': 4, 'reg_alpha': 7.605368050772253, 'reg_lambda': 7.565812542942202}. Best is trial 0 with value: 0.719422500697122.
[I 2025-10-27 09:15:47,849] Trial 2 finished with value: 0.7244889689322986 and parameters: {'n_estimators': 178, 'max_depth': 6, 'lea

Best parameters: {'n_estimators': 506, 'max_depth': 4, 'learning_rate': 0.010039759088193033, 'subsample': 0.5931751865647532, 'colsample_bytree': 0.5261346790371032, 'min_child_weight': 4, 'reg_alpha': 1.540290017799538, 'reg_lambda': 0.005508102339637805}
Best R2: 0.747


In [32]:
best_params = {'n_estimators': 506, 'max_depth': 4, 'learning_rate': 0.010039759088193033, 'subsample': 0.5931751865647532, 'colsample_bytree': 0.5261346790371032, 'min_child_weight': 4, 'reg_alpha': 1.540290017799538, 'reg_lambda': 0.005508102339637805}

In [33]:
# Define the select_topk function
def select_topk(X_transformed, top_idx=top_idx):
    return X_transformed[:, top_idx] if isinstance(X_transformed, np.ndarray) else X_transformed.iloc[:, top_idx]

# ---------- 4) CV vòng 2: REFIT VỚI TOP-K + THAM SỐ TỐI ƯU ----------
xgb_final = XGBRegressor(**best_params, random_state=42, n_jobs=-1, tree_method='hist')
all_rmse2, all_r2_2 = [], []

print(f"\n=== CV VÒNG 2: Top-{top_k} + Tham số tối ưu ===")
for fold, (tr_idx, te_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train, y_test = y.iloc[tr_idx], y.iloc[te_idx]

    preprocessor = make_preprocessor()
    preprocessor.fit(X_train, y_train)
    X_train_t = select_topk(apply_vt_mask(preprocessor.transform(X_train)))
    X_test_t  = select_topk(apply_vt_mask(preprocessor.transform(X_test)))

    xgb_final.fit(X_train_t, y_train)
    y_pred = xgb_final.predict(X_test_t)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    all_rmse2.append(rmse); all_r2_2.append(r2)
    print(f"Fold {fold}: RMSE={rmse:.3f}  R2={r2:.3f}")

print(f"\nFINAL: RMSE={np.mean(all_rmse2):.3f} ± {np.std(all_rmse2):.3f} | R2={np.mean(all_r2_2):.3f}")


=== CV VÒNG 2: Top-100 + Tham số tối ưu ===
Fold 1: RMSE=2.803  R2=0.698
Fold 2: RMSE=2.525  R2=0.720
Fold 3: RMSE=2.470  R2=0.778
Fold 4: RMSE=2.551  R2=0.750
Fold 5: RMSE=2.237  R2=0.790

FINAL: RMSE=2.517 ± 0.181 | R2=0.747
